## Problem 4:
<ol>
<li>Import orders table from mysql as text file to the destination /user/cloudera/problem5/text. Fields should be terminated by a tab character ("\t") character and lines should be terminated by new line character ("\n"). </li>
<li>Import orders table from mysql  into hdfs to the destination /user/cloudera/problem5/avro. File should be stored as avro file.</li>
<li>Import orders table from mysql  into hdfs  to folders /user/cloudera/problem5/parquet. File should be stored as parquet file.</li>
    
<br>  
<font color=red>**I need to figure out the answer for the both questions as it deals with Sqoop and Flume**</font>

### 4. Transform/Convert data-files at /user/cloudera/problem5/avro and store the converted file at the following locations and file formats
<ol>
<li>save the data to hdfs using snappy compression as parquet file at /user/cloudera/problem5/parquet-snappy-compress</li>
<li>save the data to hdfs using gzip compression as text file at /user/cloudera/problem5/text-gzip-compress</li>
<li>save the data to hdfs using no compression as sequence file at /user/cloudera/problem5/sequence</li>
<li>save the data to hdfs using snappy compression as text file at /user/cloudera/problem5/text-snappy-compress</li>
</ol>

<font color=blue>Let's first start the pyspark with databricks package for the avro file format</font>
<br>  
`pyspark --packages com.databricks:spark-avro_2.11:4.0.0`
<br>  
<font color=blue>The package picked was based on the Scala and Spark Version installed on your system</font>

<font color=blue>Let's create a schema for Products Data</font>

`from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType`

`prod_schema = StructType([
 StructField("product_int", IntegerType(), True), \
 StructField("product_category_id", IntegerType(), True), \
 StructField("product_name", StringType(), True), \
 StructField("product_description", StringType(), True), \
 StructField("product_price", FloatType(), True), \
 StructField("product_image", StringType(), True) ])`

<font color=blue>Now we can create a products dataframe with the new schema and the location from hdfs</font>

`products_df = spark.read.format("csv").schema(prod_schema).load("/user/retail_db/products/part-*")`

<font color=blue>Write the Dataframe to HDFS in Avro file format</font>
<br>  
`products_df.write.format("com.databrick.spark.avro").option("mode, overwrite").option("compresssion","snappy").save("/user/cca-problem/problem5/avro-snappy/")`
<br>  
`products_avro = spark.read.format("com.databricks.spark.avro").load("/user/cca-problem/problem5/avro-snappy/part-*")`
<br>  

<font color=blue>Lets validate if the file has been written to HDFS</font>

`import os`
`os.system("hdfs dfs -ls /user/cca-problem/problem5/avro-snappy/")`
<br>  
<font color=blue>Create the Products Dataframe based on the AVRO format</font>
`prod_avro = spark.read.format("com.databricks.spark.avro").load("/user/cca-problem/problem5/avro-snappy/part-*")`
<br>  
<font color=green>i. save the data to hdfs using snappy compression as parquet file at /user/cloudera/problem5/parquet-snappy-compress</font>
<br>  
`prod_parq_snappy = prod_avro.write.format("parquet").option("compression", "snappy").save("/user/cca-problem/problem5/parquet-snappy-compress/")`
<br>  
`prod_parquet = spark.read.format("parquet").load("/user/cca-problem/problem5/parquet-snappy-compress/part-*")`
***
<br>  
<font color=green>ii. save the data to hdfs using gzip compression as text file at /user/cloudera/problem5/text-gzip-compress</font>
<br>  
`from pyspark.sql.functions import concat_ws`
<br>  
`codec = "org.apache.hadoop.io.compression.GzipCodec"`
<br>  
`prod_parq_snappy = prod_avro.select(concat_ws("\t",*prod_avro.columns)).rdd.map(lambda rec: (rec[0])).saveAsTextFile("/user/cca-problem/problem5/text-gzip-compress/", codec)`
<br>  
<font color=green>iii. save the data to hdfs using no compression as sequence file at /user/cloudera/problem5/sequence</font>
<br>  
***
<ol>
<li>org.apache.hadoop.io.compress.DefaultCodec</li>
<li>org.apache.hadoop.io.compress.GzipCodec</li>
<li>org.apache.hadoop.io.compress.BZip2Codec</li>
<li>org.apache.hadoop.io.compress.DeflateCodec</li>
<li>org.apache.hadoop.io.compress.SnappyCodec</li>
<li>org.apache.hadoop.io.compress.Lz4Codec</li>
</ol>

`prod_seq_nocompress = prod_avro.select(prod_avro[0]), concat_ws("\t", *prod_avro.columns)).rdd.map(lambda rec: (int(rec[0]), rec[1])).saveAsSequenceFile("/user/cca-problem/problem5/sequence/")`
<br>  
<font color=green>It can be read by the below form:</font>

`prod_sequence = sc.sequenceFile("/user/cca-problem/problem5/sequence/part-*").toDf()`
  
***
<font color=blue>Lets validate if the file has been write to HDFS in Parquet format</font>
<br>  
`os.system("hdfs dfs -ls /user/cca-problem/problem5/parquet-snappy/")`
***

### 5. Transform/Convert data-files at /user/cloudera/problem5/parquet-snappy-compress and store the converted file at the following locations and file formats
<ol>
<li>save the data to hdfs using no compression as parquet file at /user/cloudera/problem5/parquet-no-compress</li>
<li>save the data to hdfs using snappy compression as avro file at /user/cloudera/problem5/avro-snappy</li>
</ol>

<font color=blue>The below dataframe can be utilized to perform the next two tasks</font>
<br>  
`prod_parquet` 

<font color=green>**1. save the data to hdfs using no compression as parquet file at /user/cloudera/problem5/parquet-no-compress**</font>
<br>  
`prod_parquet.write.format("parquet").option("compression","None").option("mode","overwrite").save("/user/cca-practice/problem5/parquet-no-compress/")`
<br>  
<font color=green>**2. save the data to hdfs using snappy compression as avro file at /user/cloudera/problem5/avro-snappy**</font>
<br>  
`prod_parquet.write.format("com.databricks.spark.avro").option("compression","snappy").option("mode","overwrite").save("/user/cca-practice/problem5/avro-snappy/")`
<br>  
<font color=blue>Please be noted that the default compression for Avro is Snappy, so you might need to declare the compression</font>
<br>  

### 6. Transform/Convert data-files at /user/cloudera/problem5/avro-snappy and store the converted file at the following locations and file formats
<ol>
<li>save the data to hdfs using no compression as json file at /user/cloudera/problem5/json-no-compress</li>
<li>save the data to hdfs using gzip compression as json file at /user/cloudera/problem5/json-gzip</li>
</ol>

<font color=green>**1. save the data to hdfs using no compression as json file at /user/cloudera/problem5/json-no-compress**</font>
<br>  
`prod_avro.write.format("json").option("compression","uncompressed").option("mode","PERMISSIVE").save("/user/cca-problem/problem5/json-ext/")`
<br>  

<font color=green>**2. save the data to hdfs using gzip compression as json file at /user/cloudera/problem5/json-gzip**</font>
<br>  
`prod_avro.write.format("json").option("compression","gzip").option("mode","PERMISSIVE").save("/user/cca-problem/problem5/json-gzip/")`

`prod_json_gzip = spark.read.format("json").load("/user/cca-problem/problem5/json-gzip/part-*")`
<br>  

### 7. Transform/Convert data-files at  /user/cloudera/problem5/json-gzip and store the converted file at the following locations and file formats

<font color=green>**1. save the data to as comma separated text using gzip compression at   /user/cloudera/problem5/csv-gzip**</font>
<br>  
`prod_json_gzip.write.format("csv").option("compression","gzip").option("mode","overwrite").save("/user/cca-problem/problem5/csv-gzip/")`
<br>  

### 8. Using spark access data at /user/cloudera/problem5/sequence and stored it back to hdfs using no compression as ORC file to HDFS to destination /user/cloudera/problem5/orc 

<font color=blue>**i came up with below as a solution, but i am not sure if its correct answer**</font>
<br>  
`prod_seq.write.format("orc").option("compression","uncompressed").save("/user/cca-problem/problem5/orc-file/")`


<font color=orange>**End of the Notebook**</font>